In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [2]:
hyp_btrain = pd.read_csv("./storage_basis/normal_train_embedded_jets.csv", header=None)

In [4]:
scale = lambda A: (A - np.mean(A, axis=0)) / np.std(A, axis=0)

In [5]:
labels = pd.read_csv("./normal_trailabels.csv", header=None).astype(int)


In [6]:
X_t = scale(hyp_btrain.values)

In [8]:
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor


/home/josemm/anaconda3/envs/envJaxLips/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import tqdm
import torch

In [21]:
tabnet_params = {
                 "optimizer_fn":torch.optim.Adam,
                 "device_name":'cpu',
                }

clf = TabNetClassifier(**tabnet_params
                      )

/home/josemm/anaconda3/envs/envJaxLips/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_v, y_train, y_v = train_test_split(hyp_btrain.values, labels.values, test_size=0.2, random_state=42)

In [23]:
clf.fit(
 X_train, y_train.reshape(1 , -1).squeeze(),
 eval_set=[(X_v, y_v.reshape(1 , -1).squeeze())],
 eval_metric=['auc'],
)

epoch 0  | loss: 0.24548 | val_0_auc: 0.97312 |  0:01:26s
epoch 1  | loss: 0.20154 | val_0_auc: 0.97574 |  0:02:52s
epoch 2  | loss: 0.19216 | val_0_auc: 0.97792 |  0:04:17s
epoch 3  | loss: 0.18739 | val_0_auc: 0.97812 |  0:05:43s
epoch 4  | loss: 0.18606 | val_0_auc: 0.9785  |  0:07:08s
epoch 5  | loss: 0.18374 | val_0_auc: 0.97937 |  0:08:33s
epoch 6  | loss: 0.18369 | val_0_auc: 0.97918 |  0:09:58s
epoch 7  | loss: 0.18317 | val_0_auc: 0.9796  |  0:11:23s
epoch 8  | loss: 0.18225 | val_0_auc: 0.97909 |  0:12:48s
epoch 9  | loss: 0.18048 | val_0_auc: 0.98005 |  0:14:13s
epoch 10 | loss: 0.18008 | val_0_auc: 0.98014 |  0:15:40s
epoch 11 | loss: 0.17898 | val_0_auc: 0.97955 |  0:17:19s
epoch 12 | loss: 0.17888 | val_0_auc: 0.97984 |  0:18:58s
epoch 13 | loss: 0.17808 | val_0_auc: 0.9804  |  0:20:35s
epoch 14 | loss: 0.17871 | val_0_auc: 0.97996 |  0:22:14s
epoch 15 | loss: 0.17825 | val_0_auc: 0.98013 |  0:23:54s
epoch 16 | loss: 0.17718 | val_0_auc: 0.98034 |  0:25:43s
epoch 17 | los

/home/josemm/anaconda3/envs/envJaxLips/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [24]:
# save tabnet model
saving_path_name = "./tabnet_model_test_1"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at ./tabnet_model_test_1.zip


# Valid data data

In [25]:
# define new model with basic parameters and #load state dict weights
#loaded_clf = TabNetClassifier()#
#loaded_clf.load_model(saved_filepath)

In [26]:
hyp_bval = pd.read_csv("./storage_basis/normal_val_embedded_jets.csv", header=None)
#lab_bval = pd.read_csv("./storage_basis/beam_val_embedded_jets.csv", header=None)
val_labels = pd.read_csv("./normal_val_labels.csv", header=None).astype(int)

In [28]:
val_preds = clf.predict(hyp_bval.values)

In [34]:
1 - np.mean(val_preds - val_labels.values.reshape(1 , -1).squeeze())

0.9815855622470527

# Test

In [37]:
test_basis = pd.read_csv("./normal_test_basis.csv", header=None)
test_labels = pd.read_csv("./test_labels.csv", header=None).astype(int)

In [38]:
test_preds = clf.predict(test_basis.values)

In [39]:
1 - np.mean(test_preds - test_labels.values.reshape(1 , -1).squeeze())

0.9827077923845969

In [40]:
test_probas = clf.predict_proba(test_basis.values)